In [1]:
## EXAMPLE ON PYBULLET

import pybullet as p
import time
import pybullet_data
import numpy as np

"""
physicsClient = p.connect(p.GUI)  # or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # optionally
p.setGravity(0, 0, -10)
planeId = p.loadURDF("plane.urdf")
startPos = [0, 0, 1]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
boxId = p.loadURDF("r2d2.urdf", startPos, startOrientation)
# set the center of mass frame (loadURDF sets base link frame)
for i in range(10000):
    p.stepSimulation()
    time.sleep(1.0 / 240.0)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos, cubeOrn)
p.disconnect() """

## IMPLEMENTATION OF ACROBOT

#   SET THE LENGHT AND HEIGHT OF ACROBOT
lenghFirstLink = 1.0
lenghtSecondLink = 1.0

#   SET THE MASS OF THE FIRST LINK
massFirstLink = 1.0
#   SET THE MASS OF THE SECOND LINK
massSecondLink = 1.0
#   GRAVITY ACCELLERATION
g = 9.81


# DEFINING THE DYNAMIC OF THE SYSTEM INTO A FUNCTION
def dynamic(state, action):
    theta1, theta2, theta1dot, theta2dot = state
    tau = action

    c1 = np.cos(theta1)
    s1 = np.sin(theta1)
    c2 = np.cos(theta2)
    s2 = np.sin(theta2)
    c12 = np.cos(theta1 + theta2)
    s12 = np.sin(theta1 + theta2)

    #   MASS MATRIX
    massMatrix = np.zeros((2, 2))
    # ** is an elevation to 2
    massMatrix[0, 0] = (
        (massFirstLink + massSecondLink) * lenghFirstLink**2
        + massSecondLink * lenghtSecondLink**2
        + 2 * massSecondLink * lenghFirstLink * lenghtSecondLink * c2
    )
    massMatrix[0, 1] = (
        massSecondLink * lenghtSecondLink**2
        + massSecondLink * lenghFirstLink * lenghtSecondLink * c2
    )
    massMatrix[1, 0] = massMatrix[0, 1]
    massMatrix[1, 1] = massSecondLink * lenghtSecondLink**2

    # Coriolis and centrifugal forces
    coriolisForce = np.zeros((2, 2))
    coriolisForce[0, 0] = (
        -2 * massSecondLink * lenghFirstLink * lenghtSecondLink * s2 * theta2dot
    )
    coriolisForce[0, 1] = (
        -massSecondLink * lenghFirstLink * lenghtSecondLink * s2 * theta2dot
    )
    coriolisForce[1, 0] = (
        massSecondLink * lenghFirstLink * lenghtSecondLink * s2 * theta1dot
    )
    coriolisForce[1, 1] = 0

    # Gravity forces
    gravityForce = np.zeros((2, 1))
    gravityForce[0, 0] = (
        massFirstLink + massSecondLink
    ) * g * lenghFirstLink * s1 + massSecondLink * g * lenghtSecondLink * s12
    gravityForce[1, 0] = massSecondLink * g * lenghtSecondLink * s12

    # Compute accelerations
    # @ is an operator for matrix multiplication
    tau = np.clip(tau, -10, 10)  # Clip action to prevent large torque values
    accelleration = np.linalg.inv(massMatrix) @ (
        tau - coriolisForce @ np.array([[theta1dot], [theta2dot]]) - gravityForce
    )

    return np.array([theta1dot, theta2dot, accelleration[0, 0], accelleration[1, 0]])

    

# Control policy (swing-up and stabilization)
def control_policy(state, t):
    theta1, theta2, theta1_dot, theta2_dot = state

    # Swing-up control
    if abs(theta1) < np.pi / 2 and abs(theta2) < np.pi / 2:
        return -0.5 * theta1_dot - 1.5 * theta2_dot  # Swing-up torque

    # Stabilization control
    return (
        -10 * theta1 - 5 * theta1_dot - 5 * theta2 - 3 * theta2_dot
    )  
    # Stabilization torque


# Connect to the physics engine
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -g)

# -----------------------------------------------------------------
# Create the Acrobot in the simulation
link1_radius = 0.1
link2_radius = 0.1
link1_height = lenghFirstLink
link2_height = lenghtSecondLink

# Create the floor
planeId = p.loadURDF("plane.urdf")

# Create the base of the Acrobot
base_pos = [0, 0, 0]
base_orient = p.getQuaternionFromEuler([0, 0, 0])
baseId = p.loadURDF("acrobot_total.urdf", base_pos, base_orient)

# -----------------------------------------------------------------

# LOAD ACROBOT
acrobot_id = p.loadURDF("acrobot_total.urdf")

# Enable motors for the joints
p.setJointMotorControl2(acrobot_id, 0, p.POSITION_CONTROL, targetPosition=0, force=0)
p.setJointMotorControl2(acrobot_id, 1, p.VELOCITY_CONTROL, force=0)


# Initial state
theta1_0 = np.random.uniform(low=-np.pi, high=np.pi)
theta2_0 = np.random.uniform(low=-np.pi, high=np.pi)
theta1_dot_0 = np.random.uniform(low=-1.0, high=1.0)
theta2_dot_0 = np.random.uniform(low=-1.0, high=1.0)
state = np.array([theta1_0, theta2_0, theta1_dot_0, theta2_dot_0])

# Simulation parameters
dt = 0.01  # Time step
total_time = 10.0   # Total simulation time

# SIMULATION STEP
for _ in range(int(total_time / dt)):
    # Get the current joint angles and velocities
    joint_states = p.getJointStates(acrobot_id, [0, 1])
    print(joint_states)
    theta1, theta2 = [state[0] for state in joint_states]
    theta1_dot, theta2_dot = [state[1] for state in joint_states]
    
    # Apply the control policy to get the torque
    action = control_policy([theta1, theta2, theta1_dot, theta2_dot], _ * dt)
    
    # Apply the torque to the joints
    p.setJointMotorControl2(acrobot_id, 1, p.TORQUE_CONTROL, force=action)
    
    # Step the simulation
    p.stepSimulation()
    time.sleep(dt)

# Disconnect from the physics engine
p.disconnect()

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0004049335487969396, 0.0971840517112655, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (0.0005012802800299456, 0.02312321549592146, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.0002121107631127201, -0.17121385035423978, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.0010253018976519946, -0.1951658722894259, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.003717666636965121, -0.6461675374351503, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.006375823600167013, -0.6379576711684541, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.009113411208462853, -0.6570210259910015, (0.0, 

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.17157028009675776, -0.47388088386088617, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.17346176279165884, -0.45395584677626266, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.1752724559480784, -0.43456635754068984, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.177004144579028, -0.41560527142790826, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.1786585764142367, -0.3970636404500919, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.18023747514625565, -0.3789356956845427, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.18174219871511238, -0.3611336565256167, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.18317480028376779, -0.

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2001799076688706, -0.0006238859617404158, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20018247542301879, -0.0006162609955649761, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2001850750770681, -0.0006239169718336309, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20018764273458814, -0.0006162378048116756, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20019024256459492, -0.0006239592016280889, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20019281023614605, -0.0006162411722720131, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20019541027681514, -0.000624009760583818, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2001

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20035589499355383, -0.0006262551126687317, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20035847242011892, -0.0006185823756240355, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2003610821279751, -0.0006263298854817691, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2003636598885799, -0.000618662545154222, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20036626990801948, -0.0006264046654984514, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2003688480026967, -0.0006187427225307991, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20037145833375059, -0.0006264794529310458, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20037

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20053262010337183, -0.0006288013418200675, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2005352089028962, -0.0006213118858493678, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2005378292210328, -0.0006288763527882357, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20054041835552758, -0.0006213922787435965, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20054303898623924, -0.0006289513707994574, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2005456284557309, -0.0006214726779969026, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20054824939904622, -0.0006290263956799738, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2005

...

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20163279795969302, -0.0006381707155537233, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20163548391391248, -0.0006446290126716365, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20163814329902222, -0.0006382524263393394, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20164082957179771, -0.0006447054661207954, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20164348929739628, -0.0006383341436582018, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20164617588875636, -0.0006447819264171224, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20164883595486857, -0.0006384158669345912, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20182297444472788, -0.0006473089547574896, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20182564576236445, -0.0006411162327744327, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20182834320256288, -0.0006473856476205422, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20183101486159516, -0.000641198167747413, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20183371262137562, -0.000647462347306691, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2018363846218294, -0.0006412801089105741, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20183908270122056, -0.0006475390538794039, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2018

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2020186296973481, -0.0006436550476814172, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20202133807778744, -0.0006500113054391608, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20202402074775208, -0.0006438407915111219, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2020267295871549, -0.0006501214566772013, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20202923837386813, -0.0006021088111754407, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20203175870280404, -0.0006048789446165356, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2020344575880517, -0.0006477324594328583, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2020

((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2022155321807099, -0.0006534615253949681, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20221822272694098, -0.000645731095458147, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20222094161345106, -0.0006525327624206892, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20222363335211707, -0.0006460172798442572, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20222635288973403, -0.000652689028069178, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20222904615784926, -0.0006463843476503315, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.20223176640349214, -0.0006528589542910725, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0))
((0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0), (-0.2022

error: Not connected to physics server.